A neural network Equalizer for known channels.  

Inputs: received signal, coefficients of taps
Outputs: equalized signal
Training: received signal, orginial signal, coefficients of taps
Loss function: squared loss

In [50]:
# Construct the data: BPSK
import numpy as np
import commpy as cp
import scipy.signal as sig

# length of array of bits
n=1000

# create a random array of bits
input_data_bits=np.random.randint(0,2,(n)) 

# modulate the data bits into complex numbers
input_data_constellations = input_data_bits*2 -1


# an example of a channel function with two consecutive taps
channel_function=[0.9,0.89]
# convolution of the input complex data with the channel transfer function
channel_output = sig.convolve(input_data_constellations, channel_function, mode='full')
l=channel_output.shape[0]

# construct data matrix X with rows of history of symbols received
X=np.zeros((l,12))
for i in range (0,l):
    X[i,0]=channel_function[0]
    X[i,1]=channel_function[1]
    for j in range(0,9):
        if j<=i:
            X[i,j+2]=channel_output[i-j]

# construct the training vector y from original symbols
y=input_data_constellations
# ensure that dimension of y is the same as dimension of X, may need to append 0s to y
num_zero_pad=l-y.shape[0]
y=np.pad(y, (0, num_zero_pad), 'constant')

In [51]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [44]:
# training the model
from sklearn.neural_network import MLPRegressor
# mlp=MLPRegressor(hidden_layer_sizes=(12),activation='relu')
mlp=MLPRegressor(hidden_layer_sizes=(12), activation='tanh')
# mlp=MLPRegressor(hidden_layer_sizes=(12), activation='logistic')
# mlp=MLPRegressor(hidden_layer_sizes=(12), activation='identity')

mlp.fit(X_train,y_train)

MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=12, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [53]:
# Predictions and Evaluation
predictions=mlp.predict(X_test)
np.linalg.norm(y_test-predictions,2)


11.780571062062009

In [ ]:
# input = channel taps
# output = channel taps
# the best FIR of the channel taps